In [ ]:
#!pip install gensim
#!pip install spacy
#!pip install nltk
#!pip install pyLDAvis

In [1]:
import pandas as pd
import unicodedata
import re
import contractions
import string

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
#spacy
import spacy
from nltk.corpus import stopwords
#vis
import pyLDAvis
import pyLDAvis.gensim_models

In [17]:
def load_data(path):
    return pd.read_csv(path)
tweets_df=load_data("qatar_loc_data.csv")

In [18]:
tweets_df = tweets_df[["text","classification","country"]]
tweets_df = tweets_df.loc[tweets_df["country"]=="Germany"]

In [19]:
tweets_df = tweets_df.loc[tweets_df["classification"]=="negative"]

In [ ]:
tweets_df

In [20]:
def to_lowercase(text):
    return text.lower()
#converting every row of the column into lower case 
tweets_df.text=tweets_df.text.apply(to_lowercase)

In [21]:
def standardize_accented_chars(text):
 return unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8", "ignore")
#testing the function on a single sample for explaination
print(standardize_accented_chars('Sómě words such as résumé, café, prótest, divorcé, coördinate, exposé, latté.'))
#standardizing accented characters for every row
tweets_df.text=tweets_df.text.apply(standardize_accented_chars)

Some words such as resume, cafe, protest, divorce, coordinate, expose, latte.


In [22]:
def get_number_of_urls(documents):
    print("{:.2f}% of documents contain urls".format(sum
(documents.apply(lambda x:x.find('http'))>0)/len
(documents)*100))
# Passing the 'text' column of the dataframe as the argument
print(get_number_of_urls(tweets_df.text)) 

51.25% of documents contain urls
None


In [23]:
def remove_url(text):
 return re.sub(r"https?:\S*", "", text)
#testing the function on a single sample for explaination
print(remove_url('using https://www.google.com/ as an example'))
#removing urls from every row
tweets_df.text=tweets_df.text.apply(remove_url)

using  as an example


In [24]:
def expand_contractions(text):
    expanded_words = [] 
    for word in text.split():
       expanded_words.append(contractions.fix(word)) 
    return " ".join(expanded_words)
#testing the function on a single sample for explaination
print(expand_contractions("Don't is same as do not"))
#expanding contractions for every row
tweets_df.text=tweets_df.text.apply(expand_contractions)

Do not is same as do not


In [25]:
def remove_mentions_and_tags(text):
    return re.sub(r"@\S*", "", text)
    #return re.sub(r"#\S*", "", text)
#testing the function on a single sample for explaination
print(remove_mentions_and_tags('Some random @abc and #def'))
#removing mentions and tags from every row
tweets_df.text=tweets_df.text.apply(remove_mentions_and_tags)

Some random  and #def


In [26]:
def keep_only_alphabet(text):
    return re.sub(r"[^a-z]", " ", text)
#testing the function on a single sample for explaination
print(keep_only_alphabet('Just a bit more $$processing #required.Just a bit!!!'))
#for all the rows
tweets_df.text=tweets_df.text.apply(keep_only_alphabet)

 ust a bit more   processing  required  ust a bit   


In [27]:
def remove_stopwords(text, nlp, custom_stop_words=None, remove_small_tokens=True, min_len=2):
    # If custom stop words are provided, add them to the default stop words list
    if custom_stop_words:
        nlp.Defaults.stop_words |= set(custom_stop_words)

    filtered_tokens = [] 
    doc = nlp(text)
    for token in doc:
        if not token.is_stop:
            # If small tokens have to be removed, select only those which are longer than min_len 
            if remove_small_tokens and len(token.text) <= min_len:
                continue
            filtered_tokens.append(token.text)

    # If after the stop word removal, words are still left in the sentence, then return the sentence as a string, else return None 
    return ' '.join(filtered_tokens) if filtered_tokens else ""

# Download spaCy model 'en_core_web_sm'
!python -m spacy download en_core_web_sm

# Creating a spaCy object
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 435.7 kB/s eta 0:00:30
     --------------------------------------- 0.0/12.8 MB 435.7 kB/s eta 0:00:30
     --------------------------------------- 0.0/12.8 MB 217.9 kB/s eta 0:00:59
     --------------------------------------- 0.0/12.8 MB 217.9 kB/s eta 0:00:59
     --------------------------------------- 0.1/12.8 MB 286.7 kB/s eta 0:00:45
     --------------------------------------- 0.1/12.8 MB 286.7 kB/s eta 0:00:45
     --------------------------------------- 0.1/12.8 MB 286.7 kB/s eta 0:00:45
     --------------------------------------- 0.1/12.8 MB 209.5 kB/s eta 0:01:01
     --------------------------------------- 0.1/12.8 MB 209.5 kB/s eta 

In [28]:
#removing stop-words and short words from every row
tweets_df.text=tweets_df.text.apply(lambda x:remove_stopwords(x,nlp,{"rt ","fifaworldcup","world","cup","worldcup","qatar"}))

In [29]:
tweets_df.loc[tweets_df["text"]==None]

,text,classification,country


In [30]:
def lemmatize(text, nlp):
   doc = nlp(text)
   lemmatized_text = []
   for token in doc:
       lemmatized_text.append(token.lemma_)
   return " ".join(lemmatized_text)
#testing the function on a single sample for explaination
print(lemmatize("Reading NLP blog is fun." ,nlp ))
#Performing lemmatization on every row
tweets_df.text=tweets_df.text.apply(lambda x:lemmatize(x,nlp))

read NLP blog be fun .


In [31]:
def generate_tokens(tweet):
    words=[]
    for word in tweet.split(" "):
    # using the if condition because we introduced extra spaces during text cleaning
        if word!="":
           words.append(word)
        return words
#storing the generated tokens in a new column named 'words'
tweets_df['tokens']=tweets_df.text.apply(generate_tokens)

In [32]:
def create_dictionary(words):
    return corpora.Dictionary(words)
#passing the dataframe column having tokens as the argument
id2word=create_dictionary(tweets_df.tokens)
print(id2word)

Dictionary<1756 unique tokens: ['great', 'half', 'nation', 'mad', 'israelis']...>


In [39]:
def create_document_matrix(tokens,id2word):
    corpus = []
    for text in tokens:
       corpus.append(id2word.doc2bow(text))
    return corpus
#passing the dataframe column having tokens and dictionary
corpus=create_document_matrix(tweets_df.tokens,id2word)

In [40]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
 id2word=id2word,
 num_topics=10,
 random_state=100,
 )

In [45]:
def get_lda_topics(model, num_topics, top_n_words):
    word_dict = {}
    for i in range(num_topics):
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in model.show_topic(i, topn=top_n_words)]

    return pd.DataFrame(word_dict)

get_lda_topics(lda_model,6,10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06
0,fifa,basically,force,good,second,probably
1,know,game,germany,football,end,president
2,terrible,think,player,thank,fuck,country
3,shit,time,death,yeah,security,migrant
4,boycott,arab,like,video,see,big
5,course,go,fail,threaten,disgraceful,human
6,year,give,say,get,hey,bribe
7,qatar,tell,absolutely,focus,watch,day
8,major,interesting,buy,man,thing,way
9,glad,yes,international,love,imagine,reporter


In [46]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

C:\Users\nikka\anaconda3\envs\Youtube\lib\site-packages\sklearn\manifold\_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.327094 -0.207416       1        1  11.859013
1     -0.045406  0.323837       2        1  11.708436
7     -0.300445  0.233389       3        1  10.178813
9      0.194131 -0.239278       4        1  10.075899
8      0.339771 -0.060178       5        1   9.792560
3      0.249886  0.121943       6        1   9.624956
2      0.119196  0.219125       7        1   9.391027
4     -0.021582 -0.318733       8        1   9.353345
6     -0.187242  0.025027       9        1   9.146053
5     -0.021213 -0.097714      10        1   8.869898, topic_info=           Term        Freq       Total Category  logprob  loglift
317     iranian  313.000000  313.000000  Default  30.0000  30.0000
126  journalist  226.000000  226.000000  Default  29.0000  29.0000
12         fifa  257.000000  257.000000  Default  28.0000  28.0000
160        good  177.000000  177.000000  Default  27.0000  27.0000
313        know  170.000000  170.000000  Default  26.0000  26.0000
..          ...         ...         ...      ...      ...      ...
281        hype    0.946325    2.422611  Topic10  -6.9007   1.4825
10         film    1.075544    3.343596  Topic10  -6.7727   1.2883
53    criticism    3.511210   16.730516  Topic10  -5.5896   0.8612
12         fifa    2.405516  257.312654  Topic10  -5.9678  -2.2500
48     football    1.518750   64.635131  Topic10  -6.4276  -1.3284

[345 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
152       2  0.038496  absolutely
152       7  0.923914  absolutely
95        7  0.832193      accuse
181       6  0.925572    activist
215       4  0.895925    actually
...     ...       ...         ...
480       8  0.360507         yea
174       6  0.959892        yeah
177       1  0.929650        year
285       2  0.935205         yes
447       7  0.789628       young

[388 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 8, 10, 9, 4, 3, 5, 7, 6])